In [1]:
import numpy as np
import os
import torchvision.transforms as transforms
from models import *
from PIL import Image, ImageOps
from imageio import imread
from pathlib import Path
from matplotlib import pyplot as plt

os.environ['cuda_visible_devices'] = '0'

In [2]:
if torch.cuda.is_available():
    print("gpu found")
    device = 'cuda:0'
else:
    device = 'cpu'

In [3]:
torch.cuda.empty_cache()

In [4]:
import gc
gc.collect()

0

In [5]:
model = ResNet_extractor().to(device)
data_transform = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

/opt/homebrew/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
feature_dir = '../features/'
if os.path.exists(feature_dir):
    print("Adding to feature directory...")
else:
    os.mkdir(feature_dir)

Adding to feature directory...


In [7]:
patient_info_file = '../patient_id_all.txt'
with open(patient_info_file, 'r') as f:
    patient_list = f.read().splitlines()
patient_list

['TCGA-DK-A2I6-01']

In [8]:
root_dir = Path('../tile_data/')
save_dir = '../features/'
batch = 64
feature_list = []
# with torch.cuda.device(0):
for patient in patient_list:
    patient_tile_dir = root_dir/patient
    file = [x for x in patient_tile_dir.iterdir() if x.is_file()]
    file.sort()
    imgs = []
    for i in range(0, (len(file)//batch)+1, 1):
        img_batch = []
        start = i*batch
        end = (i+1)*batch if (i+1)*batch < len(file) else len(file)
        for j in range(start, end, 1):
            img = Image.open(file[j]).convert('RGB') 
            img = data_transform(img).unsqueeze(0) # 1, 3, 224, 224
            img_batch.append(img)
        images = torch.concat(img_batch, dim=0)
        print("images shape is ", images.shape)
#         assert images.shape[0] == batch
        features = model(images.to(device))
        print(features.shape)
        feature_list.append(features.detach().cpu().numpy())
        del features
    feature_list = np.concatenate(feature_list, axis=0)
    dest = os.path.join(save_dir, patient+'.npy')
    np.save(dest, feature_list)
    

images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images shape is  torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 1, 1])
torch.Size([64, 2048])
images sha

In [10]:
feature_list.shape

(4997, 2048)

In [20]:
# dest = os.path.join(save_dir, patient+'.npy')
# np.save(dest, n)